In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import Dense, LSTM, Dropout
from keras.models import Sequential

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

colnames = ['isitu-LST',
            'Band1','Band2','Band3','Band4','Band6','Band7','Band8','Band9',
            'Band10' ,'Band11','Band12','Band13','Band14','Band15','Band16',
            '30daysBand3','30daysBand13',
            'SolarZA','SateZA','ESR','Height',
            'LandType',
            'insitu-HM','insitu-TD', 
            'month', 'time']

def load_data(filename):  # 필요에 따라 레이블을 사용할수도 안할수도 있음
    df = pd.read_csv(filename)[colnames]

    X = df.loc[:, colnames[1:]].values
    y = df.loc[:, colnames[0]].values  # 레이블 값을 지난번과 다르게 0~8까지로 사용하도록 변경

    return X, y



X, y = load_data("merge_data_56789.csv")

# X, y = load_data("/content/drive/MyDrive/wheather/data/과제_최종_1/과제2 결측치 제거.csv")

# X, y = load_data("/content/drive/MyDrive/wheather/data/remove_outlier.csv")
# X, y = load_data("/content/drive/MyDrive/wheather/data/과제2 결측제거/과제2 결측치 제거.csv")
# X, y = load_data("/content/drive/MyDrive/wheather/data/과제2 결측제거 5689/과제2 결측치 제거.csv")
# X, y = load_data("/content/drive/MyDrive/wheather/data/과제2 결측제거 extract_col/과제2 결측치 제거.csv")

print(X.shape, y.shape)

(4176440, 26) (4176440,)


In [3]:
y.min(), y.max()

model = Sequential()

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3)
checkpoint = tf.keras.callbacks.ModelCheckpoint("lstm01.h5", monitor="val_loss", verbose=1 , save_best_only=True, mode="auto")

Adam = tf.keras.optimizers.Adam(clipnorm=1.)

model = Sequential()

# model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(25, int(len(colnames)-1))))
# # model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Flatten())
# model.add(RepeatVector(64))

# model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(64, int(len(colnames)-1))))
# model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Flatten())
# model.add(RepeatVector(64))

model.add(LSTM(256, input_shape=(int(len(colnames)-1), 1), return_sequences=True)) #input_shape은 x의 라벨값 6개 시퀀스 출력은 True 512차원 출력
model.add(Dropout(0.5)) #과적합 방지를 위한 드랍아웃 비율은 0.5
model.add(LSTM(128, return_sequences=True)) #LSTM 층  256차원출력+
model.add(Dropout(0.5)) #드랍아웃 층
model.add(LSTM(64, return_sequences=True)) #LSTM 층  256차원출력+
model.add(Dropout(0.5)) #드랍아웃 층
# model.add(LSTM(32)) #LSTM층 128차원 출력
# model.add(Dropout(0.5)) #과적합 방지를 위한 드랍아웃 비율은 0.5
model.add(Dense(100)) #활성화 함수
model.add(Flatten())
model.add(Dense(1, activation='relu')) #활성화 함수

# model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mse'])
model.compile(loss='mean_squared_error', optimizer=Adam, metrics=['mse'])
model.summary()

model.fit(X, y, epochs=100, batch_size=1024, verbose=1, callbacks=[early_stopping, checkpoint])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 26, 256)           264192    
                                                                 
 dropout_4 (Dropout)         (None, 26, 256)           0         
                                                                 
 lstm_5 (LSTM)               (None, 26, 128)           197120    
                                                                 
 dropout_5 (Dropout)         (None, 26, 128)           0         
                                                                 
 lstm_6 (LSTM)               (None, 26, 64)            49408     
                                                                 
 dropout_6 (Dropout)         (None, 26, 64)            0         
                                                                 
 dense_2 (Dense)             (None, 26, 100)          

2022-08-01 08:27:39.103441: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-01 08:27:39.477218: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-01 08:27:39.542351: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-01 08:27:39.632828: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-01 08:27:40.401381: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-01 08:27:40.487711: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-01 08:27:40.577635: I tensorflow/core/grappler/optimizers/cust

4079/4079 [==============================] - 680s 166ms/step - loss: 11.7123 - mse: 11.7123
Epoch 2/100
4079/4079 [==============================] - 674s 165ms/step - loss: 6.7791 - mse: 6.7791
Epoch 3/100
4079/4079 [==============================] - 669s 164ms/step - loss: 6.3818 - mse: 6.3818
Epoch 4/100
4079/4079 [==============================] - 695s 170ms/step - loss: 5.9971 - mse: 5.9971
Epoch 5/100
4079/4079 [==============================] - 727s 178ms/step - loss: 5.6254 - mse: 5.6254
Epoch 6/100
4079/4079 [==============================] - 729s 179ms/step - loss: 5.3191 - mse: 5.3191
Epoch 7/100
4079/4079 [==============================] - 729s 179ms/step - loss: 5.0396 - mse: 5.0396
Epoch 8/100
4079/4079 [==============================] - 730s 179ms/step - loss: 4.8018 - mse: 4.8018
Epoch 9/100
4079/4079 [==============================] - 730s 179ms/step - loss: 4.6578 - mse: 4.6578
Epoch 10/100
4079/4079 [==============================] - 732s 179ms/step - loss: 4.5149 - m

Exception ignored in: <generator object iter_fields at 0x41600ef20>
Traceback (most recent call last):
  File "/Users/sehunkim/miniforge3/lib/python3.9/ast.py", line 251, in iter_fields
    yield field, getattr(node, field)
KeyboardInterrupt: 

KeyboardInterrupt



In [ ]:
model.save("./pknu_lstm4_56789.h5")

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# X, y = load_data("/content/drive/MyDrive/wheather/data/only78.csv")
test_df = pd.read_csv("/content/drive/MyDrive/wheather/test/only78.csv")[colnames].astype(np.float32)
X = test_df.loc[:, colnames[1:]].values
y = test_df.loc[:, colnames[0]].values

y_score = model.predict(X)
print("MSE: " + str(mean_squared_error(y, y_score)))
print("RMSE: " + str(np.sqrt(mean_squared_error(y, y_score))))
print("MAE: " + str(mean_absolute_error(y, y_score)))
# y_pred = np.argmax(y_score, axis=1)
# pd.Series(y_pred, name=check_col[0]).to_csv("LSTM_result.csv")

In [ ]:
import pandas as pd


extract_col = ['YearMonthDayHourMinute', 'isitu-LST','insitu-TG','Band1','Band2','Band3','Band4','Band6','Band7','Band8','Band9','Band10','Band11','Band12','Band13','Band14','Band15','Band16','30daysBand3','30daysBand13','SolarZA','SateZA','ESR','Height','LandType','insitu-HM','insitu-TD']

df = pd.read_csv("/content/drive/MyDrive/wheather/data/과제2 결측치 제거_3.2g.csv")[extract_col]
df['month'] = pd.to_datetime(df['YearMonthDayHourMinute'], format='%Y%m%d%H%M%S').dt.strftime('%m').astype(int)
extract_col.append('month')

# df['LandType'] += 1

df.to_csv("/content/drive/MyDrive/wheather/data/과제2 결측치 제거_3.2g_2.csv")